#一、项目背景介绍

在作业准备初期，想要接触一下跟踪或者分割算法的训练。
最终选择了分割方面的内容，以及车道线分割的内容。
车道线包括多种类型，并且车道线的分割是目前热门的智能驾驶的一部分，我们在检测并分割出不同类型的车道线后，才能完成车辆的循线驾驶工作。

#二、数据介绍

在数据集方面使用了飞桨的公开数据集[VIL100](http://aistudio.baidu.com/aistudio/datasetdetail/115234)

三、模型介绍
在本次作业中，主要使用paddledetection的paddleseg已有框架中的deeplabv3模型进行了训练。
首次使用，因此在api安装及调用的过程中遇到了一些问题。比如，requirements安装过程中，遇到了包的版本不匹配徐重新安装；某个包需要user的授权才可安装的问题等；更加低级的问题是，在文件链接中加入了地址，而没有调整运行目录，导致报错。

In [ ]:
#数据集加载
!unzip -q /home/aistudio/data/data115234/VIL100.zip 
!unzip -q /home/aistudio/data/data80207/PaddleSeg.zip


In [ ]:
#数据可视化准备
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#根目录设定
ROOT = "VIL100"
#读取训练集、测试集，并展示其数量
#with open("路径", "mode") as 文件名:  mode:'r' read,'w'write, 'a'追加
with open("VIL100/data/train.txt", "r") as TRAIN:   
    trainData = TRAIN.readlines()  #readlines读取整个文件，返回列表；read读取整个文件，返回字符串
    print('训练集数据量: {}'.format(len(trainData)))

with open("VIL100/data/test.txt", "r") as TEST: 
    testDatas = TEST.readlines()
    print('测试集数据量: {}'.format(len(testDatas)))

In [ ]:
#安装paddleseg，先安装补充包，再安装PaddleSeg
!pip install -r requirements.txt
!python setup.py install

数据集列表准备

In [ ]:
from paddle.io import Dataset   #加载数据处理包

#训练/测试模式，读取数据
class VIL(Dataset):
    def __init__(self, mode='train'):
        super(VIL, self).__init__() 
        self.train_data_paths = self.load_train_data()# 获取训练集
        self.test_data_paths = self.load_test_data()  # 获取训练集
        self.mode = mode
        self.root = "VIL100"

    def __getitem__(self, idx):    #def __getitem__起到的作用是展示取到的数据
        if self.mode == 'test':
            data_paths = self.test_data_paths
        else:
            data_paths = self.train_data_paths

        image = cv2.imread(self.root + data_paths[idx].split(" ")[0])  #图片文件名称
        image = (image / 255. * 2. - 1.).astype('float32')   #数据类型强制转换为浮点型
        image = np.transpose(image, (2, 0, 1))
        label = cv2.imread(self.root + data_paths[idx].split(" ")[1])   #标签文件名称
        label = label.astype('float32')   #数据类型转换
        label = np.transpose(label, (2, 0, 1))

        return image, label

    def __len__(self):
        if self.mode == 'test':
            return len(self.test_data_paths)
        else:
            return len(self.train_data_paths)

    @staticmethod
    def load_train_data():
        data_path = 'VIL100/data/train.txt'
        with open(data_path, "r") as trainList: 
            return trainList.readlines()

    @staticmethod
    def load_test_data():
        data_path = 'VIL100/data/test.txt'
        with open(data_path, "r") as testList: 
            return testList.readlines()

traindataset = VIL('train')
testdataset  = VIL('test')

!python tools/create_dataset_list.py


四、模型训练

In [ ]:
from paddleseg.models import DeepLabV3
model = DeepLabV3(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)

In [ ]:
!python train.py --config configs/deeplabv3/deeplabv3_resnet50_os8_cityscapes_1024x512_80k.yml

In [ ]:
#训练过程可视化
import os

results_save_path = 'data/results'   #结果保存位置
os.makedirs(results_save_path, exist_ok=True)  # 保存每个epoch的测试结果

weights_save_path = 'data/weights'   #权重保存位置
os.makedirs(weights_save_path, exist_ok=True)  # 保存模型

for epoch in range(EPOCHS):
    if (epoch+1) % 1 == 0:
        paddle.save(generator.state_dict(), os.path.join(weights_save_path, 'epoch'+str(epoch+1).zfill(3)+'.pdparams'))

五、模型评估

In [ ]:
!python val.py --config configs/deeplabv3/deeplabv3_resnet50_os8_cityscapes_1024x512_80k.yml

六、总结与升华
在本次学习中，接触到了许多新鲜的内容，开阔了视野。最重要的是提升了自己的代码能力，虽然还不能够完成自己组网，但对于训练中使用到的代码以及python包有了更多的理解，在后续还需要继续学习命令。

七、个人总结
目前在接触学习机器视觉、深度学习的阶段，小白一枚，请见谅。
GitHub及gitee链接待补，还未申请成功。GitHub项目显示未加入任何团体不能将导出的文件增加，正在搜索解决，线提交上，不好意思。
